In [1]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
from collections import namedtuple
import os
import xml.etree.ElementTree as ET

In [2]:
settings = {
    'queries_xml': '/home/subject/Downloads/web2008_adhoc.xml', # path to queries xml
    'relevance_xml': '/home/subject/Downloads/or_relevant-minus_table.xml', # path to relevance xml
    'collection_dir': '/home/subject/Documents/informational retrieval/all_info2/', # folder with contents of all_info.zip
}

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360, 'maxsize': 25}])

In [4]:
if not es.indices.exists(index='byweb'):
    es.indices.create(index='byweb')

In [5]:
index_settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            },
            'stem_content': {
                'type': 'text'
            },
            'title': {
                'type': 'text'
            },
            'id': {
                'type': 'keyword'
            },
            'url': {
                'type': 'keyword'
            }
        }
    },
    'settings': {
       'analysis': {
            'analyzer': {
                'white_lover': {
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase'
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace'
                }
            },
        }
    }
}

In [6]:
def recreate_index():
    es.indices.delete(index='byweb')
    es.indices.create(index='byweb', body=index_settings)

In [7]:
%time
recreate_index()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.11 µs


In [8]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def es_actions_generator(docs):
    for doc in os.listdir(docs):
        doc = os.path.join(docs, doc)
        with open(doc) as d:
            data = json.load(d)
            for document in data:
                yield create_es_action('byweb', document['id'], document)


In [9]:
generator = es_actions_generator(settings['collection_dir'])
for ok, result in tqdm(parallel_bulk(es, generator, queue_size=4, thread_count=4, chunk_size=1000)):
    if not ok:
        print(result)

Время работы процедуры выше: чуть больше 5 минут.

Если посмотрим сюда: "http://localhost:9200/_stats/indexing,store?pretty " и найдем:

`indices.byweb.total.store.size_in_bytes`

увидим, что размер индекса равен ~3 Гб. Размер информации, запихнутой в индекс, ~4.1 Гб.

(Конечно, все это можно было сделать программно, но посмотреть на один показатель быстрее ручками.)

In [10]:
Query = namedtuple('Query', ['query_id', 'text', 'relevant'])

In [11]:
def parse_query(query):
    text = query.find('{http://www.romip.ru/data/adhoc}querytext').text
    query_id = query.attrib['id']
    return Query(query_id=query_id, text=text, relevant=[])


def extract_queries():
    filename = settings['queries_xml']
    with open(filename) as f:
        tree = ET.parse(filename)
        root = tree.getroot()
        tasks = root.findall('{http://www.romip.ru/data/adhoc}task')
        qs = list(map(parse_query, tasks))
        queries = dict()
        for x in qs:
            queries[x.query_id] = x
        return queries

In [12]:
def extract_relevance():
    queries_dict = extract_queries()
    def parse_relevance(query):
        query_id = query.attrib['id']
        q = queries_dict[query_id]
        for doc in query.findall('./{http://www.romip.ru/common/merged-results}document'):
            if doc.attrib['relevance'] == 'vital':
                q.relevant.append(doc.attrib['id'])
        return q

    filename = settings['relevance_xml']
    with open(filename) as f:
        tree = ET.parse(filename)
        root = tree.getroot()
        tasks = root.findall('{http://www.romip.ru/common/merged-results}task')
        return list(map(parse_relevance, tasks))

In [116]:
queries = extract_relevance()
# important: -38 queries
queries = list(filter(lambda q: q.relevant, queries))
print(len(queries))

495


In [118]:
def search(query, result_size=20):
    """
    Use this function for search.
    """
    
    query = {
        'query': {
             'bool': {
                'should': 
                    {
                        'match': {
                            'content': query.text
                        }
                    }
            }
        }
    }
    
    result = es.search(index='byweb', body=query, size=result_size)
    return list(map(lambda x: x['_id'], result['hits']['hits']))

def pretty_print_result(search_result):
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='byweb', id=doc_id)['_source']

Статистические показатели.

In [136]:
def stats_at(k, query, query_result):
    relevant = query.relevant
    result_at_k = query_result[:k]
    good = set(result_at_k).intersection(relevant)
    return [len(good) / k, len(good) / len(relevant)]

In [137]:
def get_ap_at(k, query, query_result):
    s = 0
    n = 0
    real_k = min(k, len(query_result))
    for i in range(real_k):
        result = query_result[i]
        if result in query.relevant:
            s += stats_at(i + 1, query, query_result)[0]
            n += 1
    return 0 if n == 0 else s / n

In [138]:
def get_map_at(k, queries, q_results):
    s = 0
    for query, query_result in zip(queries, q_results):
        s += get_ap_at(k, query, query_result)
    return s / len(queries)

In [151]:
def get_queries_results(qs, search_fun=search, k=20):
    return [search_fun(q, k) for q in qs]

In [152]:
def get_queries_stats(search_fun, k=20):
    results = get_queries_results(queries, search_fun, k)
    
    precision = 0
    recall = 0
    r_precision = 0
    for q, r in zip(queries, results):
        stats = stats_at(k, q, r)
        precision += stats[0]
        recall += stats[1]
        r_precision += stats_at(len(q.relevant), q, r)[0]
    
    queries_size = len(queries)
    print("Средняя точность на уровне k=20 по всем запросам:", precision / queries_size)
    print("Средняя полнота на уровне k=20 по всем запросам:", recall / queries_size)
    print("Средняя R-точность на уровне k=20 по всем запросам:", r_precision / queries_size)
    print("MAP на уровне k=20:", get_map_at(k, queries, results))

In [153]:
get_queries_stats(search)

Средняя точность на уровне k=20 по всем запросам: 0.33878787878787886
Средняя полнота на уровне k=20 по всем запросам: 0.2224458420703543
Средняя R-точность на уровне k=20 по всем запросам: 0.18736074157738253
MAP на уровне k=20: 0.5054045146188249


In [154]:
%%timeit -n 3 -r 3
_ = get_queries_results(queries)

3 loops, best of 3: 3.06 s per loop


In [ ]:
# Query(query_id='arw53730', text='Ремолан', relevant=['980017']) []
# потому что mystem, ремолать

In [ ]:
# TODO: Проиндексируйте коллекцию с морфологической обработкой. (лемматизированный контент - в stem_content). 
# Сравните качество поиска с вариантом без морфологии (не забывайте, что запрос надо тоже привести к основам/леммам).

In [156]:
# Ячейка для лемматизации запроса; совпадает с лемматизацией для коллекции.

stop_words = {'г', '©'}
def get_stop_words(files):
    for file in files:
        with open(file) as f:
            for word in f:
                stop_words.add(word.split()[0])

get_stop_words(['../extractor/stopwords/english', '../extractor/stopwords/russian'])

def is_not_stop_word(d):
    return not getLexOrText(d) in stop_words


normal_word = re.compile('^[A-Za-z0-9Ѐ-ӿ]*$')
def is_normal_word(d):
    return normal_word.match(d['text']) is not None


def getText(d):
    return d['text'].lower()

def getLexOrText(d):
    if 'analysis' not in d or not d['analysis']:
        return getText(d)
 
    analysis = d['analysis'][0]
    return analysis['lex'] if 'lex' in analysis else getText(d)


from pymystem3 import Mystem
m = Mystem()

def lemmatize(query_text):
    result = m.analyze(query_text)

    result = list(filter(bool, result))
    result = list(filter(lambda x: 'analysis' in x or is_normal_word(x), result))
    json_results = list(filter(is_not_stop_word, result))

    lexed_content = " ".join(list(map(getLexOrText, json_results)))
    return lexed_content

In [157]:
def l_search(query, result_size=20):
    query_text = lemmatize(query.text)
    query = {
        'query': {
             'bool': {
                'should': 
                    {
                        'match': {
                            'stem_content': query_text
                        }
                    }
            }
        }
    }
    
    result = es.search(index='byweb', body=query, size=result_size)
    return list(map(lambda x: x['_id'], result['hits']['hits']))

In [158]:
get_queries_stats(l_search)

Средняя точность на уровне k=20 по всем запросам: 0.39828282828282874
Средняя полнота на уровне k=20 по всем запросам: 0.27147193922637175
Средняя R-точность на уровне k=20 по всем запросам: 0.22705348784596355
MAP на уровне k=20: 0.5539490841880522


In [159]:
%%timeit -n 3 -r 3
_ = get_queries_results(queries, l_search)

3 loops, best of 3: 3.52 s per loop


In [ ]:
# TODO: Рассчитайте PageRank страниц. Предложите другие статические показатели информативности страниц. 
# Скомбинируйте BM25 и статический показатель, оцените качество поиска. 